Solved Digit Recognizer problem in easy and intuitive way so to also understand a begineer .<br>
Convolutional Neural Network (CNN) is deep learning's artificial neural network used for <br>
imagery research and other object recognition . <br>

Importing important python libraries for data maniputaltion, data analysis and for <br>
performing mathematical operations .

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns

Loading training and testing dataset with pd.read_csv function from pandas

In [ ]:
training_data = pd.read_csv('../input/train.csv')
testing_data = pd.read_csv('../input/test.csv')

In [ ]:
training_data.head()

In [ ]:
testing_data.head()

Values occuring of target feature -> label

In [ ]:
label = training_data.label.value_counts()
label

In [ ]:
sns.countplot(training_data['label'])
plt.title("Distribution of feature")
plt.ylabel("Number of Occurences")
plt.xlabel("Labels")

Transforming Train_data and Test_data into images\labels.

In [ ]:
train_data = training_data.drop(['label'], axis=1).values.astype('float32') # all pixel values
target = training_data['label'].values.astype('int32') # only labels i.e targets digits
test_data = testing_data.values.astype('float32')
train_data = train_data.reshape(train_data.shape[0], 28, 28) / 255.0
test_data = test_data.reshape(test_data.shape[0], 28, 28) / 255.0

Converting label into categorical form (One Hot Encoding) <br>
Created 10 different feature columns as label were 10 

In [ ]:
import keras
num = 10
target = keras.utils.to_categorical(target,num)

Previewing first fifty images 

In [ ]:
plt.figure(figsize=(10,10))
for i in range(50):  
    plt.subplot(5, 10, i+1)
    plt.imshow(train_data[i].reshape((28,28)),interpolation='nearest')
plt.show()

Importing all the necessary keras library <br>
If not works add 'python' after tensorflow <br>
e.g from tensorflow.python.keras.models import Sequential<br>

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
num_classes = 10
input_shape = (28, 28, 1)

Splitting training set and test set via<br>
**train_test_split** function 

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train_data, target , test_size = 0.1, random_state=42)

Reshaping train, test set for making ready to go through modelling purpose

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)  
X_val = X_val.reshape(X_val.shape[0], 28, 28,1)  
test_data = test_data.reshape(test_data.shape[0], 28, 28,1)

Information of **ImageDataGenerator** can be obtained from [here](https://keras.io/preprocessing/image/#imagedatagenerator-class)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10, # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally 
        height_shift_range=0.1,  # randomly shift images vertically 
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)  # fitting X_train model 

Information of **ReduceLROnPlateau** can be obtained from [here](https://keras.io/callbacks/)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',   # quality to be monitored 
                                            patience=3,          # no of epoch with no improvement after learning rate will be reduced
                                            verbose=1,           # update message
                                            factor=0.5,          # reducing learning rate 
                                            min_lr=0.0001)       # lower bound learning rate 

Everything about the cnn layers that are [Convolutional](https://keras.io/layers/convolutional/), [MaxPooling](https://keras.io/layers/pooling/), [Activation](https://keras.io/activations/), [BatchNormalization](https://keras.io/layers/normalization/) can be found here 

**Dropout** function used to avoid overfiting of model by terminating / disabling random node while building<br>
so neurons doesnot become dependent on any neuron or to avoid correlations while learning<br>
**Flatten** used to make all the previous output into one single column to obtain the interpreted result 

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal',input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.20))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))


Compiling the model and placing the parameters that are to be displayed when model is runned / fitted .
Loss function has given paramter as **categorical_crossentropy** because we have categorical and more <br>
than 2 attributes in output .
Optimizer is a function that minimizes the loss function of a model to gain higher accuracy peek . 

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

fitting model through all the above parameters and the functions

In [ ]:
fitting = model.fit_generator(datagen.flow(X_train, Y_train, batch_size = 90),
                              epochs = 50, validation_data = (X_val, Y_val),
                              verbose = 1, callbacks = [learning_rate_reduction])

Here summary of the model can be interpreted with each layer separetly containing <br>
number of parameters it had used along with its output shape

In [ ]:
print(model.summary())

Getting the predictions for our test data that has no labels or target values 

In [ ]:
predicted_classes = model.predict_classes(test_data)

Creating csv final submission file from above prediction against image id's 

In [ ]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(predicted_classes)+1)),
                         "Label": predicted_classes})
submissions.to_csv("submission.csv", index=False, header=True)

In [ ]:
submissions.head()